In [19]:
import numpy as np


magnitude_maximum_history=[]
OpticalFlowFeatureMapHueHistory=[]
prev_gray=None
frame_height, frame_width = None,None

def cleanForOpticalFlow(contiuFlag=False):
    global magnitude_maximum_history,OpticalFlowFeatureMapHueHistory,prev_gray,frame_height, frame_width
    magnitude_maximum_history=[]
    OpticalFlowFeatureMapHueHistory=[]
    if contiuFlag:
        prev_gray=None
        frame_height, frame_width = None,None

def OpticalFlowForClip(img,guiDebug=True,plotNum=True,frameByframe=False,lowPass=-1):
    global magnitude_maximum_history,OpticalFlowFeatureMapHueHistory,prev_gray,frame_height, frame_width

    if guiDebug:
        cv2.namedWindow('Optical Flow', cv2.WINDOW_AUTOSIZE)

    if prev_gray is None:
        frame_height, frame_width = img.shape[:2] # 获取视频分辨率
        prev_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 转换为灰度图像

    else:
        frame=img
    # for frame in imgs[1:]:
        # 转换为灰度图
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # 计算光流
        flow = cv2.calcOpticalFlowFarneback(
            prev=prev_gray,  # 上一帧的灰度图像
            next=gray,  # 当前帧的灰度图像
            flow=None,  # 初始化光流矩阵，这里未使用，故为None
            pyr_scale=0.5,  # 金字塔缩放比例，越小意味着构建更多层金字塔，提高检测的精确度和鲁棒性
            levels=3,  # 金字塔层数，表示多尺度（层次）的处理
            winsize=15,  # 平均窗口大小，越大能捕捉到更高速的运动，但也平滑更多
            iterations=3,  # 每层金字塔迭代次数，迭代次数越多，计算越精确但也越慢
            poly_n=5,  # 每个像素邻域的大小，用于多项式展开，一般为5或7
            poly_sigma=1.2,  # 高斯标准差，用于平滑导数，一般与poly_n一起使用
            flags=0  # 操作标志，如设置为cv2.OPTFLOW_USE_INITIAL_FLOW则使用flow输入作为初始流估计
        )
        # 计算光流的幅度和角度
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        magnitude_maximum_history.append(np.max(magnitude))

        magnitude_scaled=cv2.normalize(magnitude, None, 0, 1, cv2.NORM_MINMAX)
        cv2.imshow("magnitude field",magnitude_scaled)

        # region 彩色featuremap
        # 生成一个全黑的特征图
        feature_map = np.zeros((magnitude.shape[0], magnitude.shape[1], 3), dtype=np.uint8)
        # 将 magnitude 映射到亮度
        feature_map[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        # 将 angle 映射到色相
        angle_in_degrees = ((angle + np.pi) * 180 / np.pi) % 180
        feature_map[..., 0] = angle_in_degrees.astype(np.uint8)
        # 设置饱和度为最大值
        feature_map[..., 1] = 255
        # 将颜色空间转换为BGR到HSV
        feature_map = cv2.cvtColor(feature_map, cv2.COLOR_HSV2BGR)
        OpticalFlowFeatureMapHueHistory.append(feature_map)
        # 显示特征图
        cv2.imshow('Feature Map', feature_map)
        # endregion

        if guiDebug:

            # 选择每隔10个像素
            step = 10
            y, x = np.mgrid[step / 2:frame_height:step, step / 2:frame_width:step].reshape(2, -1).astype(int)
            fx, fy = flow[y, x].T
            # 创建线的起点和终点
            lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
            lines = np.int32(lines + 1.5)

            # 绘制线
            vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            cv2.polylines(vis, lines, isClosed=False, color=(0, 255, 0))
            # 为每条线绘制点
            for (x1, y1), (_x2, _y2) in lines:
                cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)

            ## 画向量场的大小


            # 选择每隔40个像素
            if plotNum:
                for y in range(0, frame_height,step):
                    for x in range(0, frame_width,step):
                        mag = magnitude[y, x]
                        # 仅绘制幅度大于1的矢量大小，减少图像的拥挤
                        if mag > 1:
                            # text = f"{mag:.1f}"
                            text = f"{int(mag)}"
                            cv2.putText(vis, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)
                            # print(f"({x, y} {text})")

            # nowmag=magnitude_maximum_history[-1]
            # cv2.putText(vis,str(nowmag) , (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) if nowmag<10 else (0,0,255), 1)
            nowmag=np.sum(magnitude)
            cv2.putText(vis,str(nowmag) , (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) if nowmag<10 else (0,0,255), 1)

            # 显示结果
            cv2.imshow('Optical Flow', vis)

            # 按'q'键退出循环
            if cv2.waitKey(-1 if frameByframe else 1) & 0xFF == ord('q'):
                cv2.destroyAllWindows()


        # 更新前一帧的图像和灰度图
        prev_gray = gray

    # if guiDebug:
    #     cv2.destroyAllWindows()

    # return {
    #     "magnitude_maximum_history": magnitude_maximum_history,
    #     "OpticalFlowFeatureMapHueHistory":OpticalFlowFeatureMapHueHistory
    # }

In [2]:
import yaml

# 从YAML文件中加载裁剪信息
with open('crop_info.yaml', 'r') as yaml_file:
    crop_info = yaml.safe_load(yaml_file)


In [16]:
#读取一个文件夹下内的所有照片
import cv2
import os

# 定义要读取的文件夹路径
folder_path = r'data/1'

# 获取文件夹下所有文件的文件名
file_names = os.listdir(folder_path)

imgs2process=[]

# 遍历文件夹下的每个文件
for file_name in file_names:
    # 构建文件的完整路径
    file_path = os.path.join(folder_path, file_name)

    # 判断文件是否是图片文件（这里简单地通过文件扩展名判断
    if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        imgs2process.append(file_path)


imgsData=[]
for imgPath in imgs2process:
    imgsData.append(cv2.imread(imgPath)[crop_info['y_start']:crop_info['y_end'], crop_info['x_start']:crop_info['x_end']])

for img in imgsData:
    OpticalFlowForClip(img,frameByframe=True,plotNum=False)

cv2.destroyAllWindows()

In [23]:
from tqdm import tqdm

video_path=r"Z:\Motion_Recognition\Head Twitch\largeSlice\output_000.mp4"


imgsData=[]
# 打开视频文件
cap = cv2.VideoCapture(video_path)
# 获取视频的总帧数
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# while True:
for step in tqdm(range(total_frames)):
    ret, prev_frame = cap.read()
    if not ret:
        break

    img=prev_frame[crop_info['y_start']:crop_info['y_end'], crop_info['x_start']:crop_info['x_end']]

    OpticalFlowForClip(img,frameByframe=False)
    cv2.imwrite(f"data/OpticalFlowFeatureMap/{step}.png",OpticalFlowFeatureMapHueHistory[-1])
    OpticalFlowFeatureMapHueHistory=[]
    magnitude_maximum_history=[]

    # if step%10==0:
    #     for OpticalFlowFeatureMap in OpticalFlowFeatureMapHueHistory:
    #         cv2.imwrite(f"data/OpticalFlowFeatureMap/{step}.png",OpticalFlowFeatureMap)
    #
    #     cleanForOpticalFlow(contiuFlag=True)



100%|██████████| 35989/35989 [1:01:05<00:00,  9.82it/s]


In [11]:
del imgsData

In [24]:

cv2.destroyAllWindows()

In [75]:
threholdForMMax=15.0  # 检测阈值

# 使用enumerate函数获取列表元素和索引，并根据元素的值进行排序
sorted_indices = sorted(enumerate(OpticalFlowResult["magnitude_maximum_history"]), key=lambda x: x[1], reverse=True) # index saves at [0] value at [1]

indices_list=[]

for i,v in sorted_indices:
    if v>threholdForMMax:
        indices_list.append(i)
    else:
        break

从大到小排列的索引: [(86, 18.364204), (89, 18.03031), (88, 16.813759), (87, 15.731115), (94, 14.637705), (90, 12.583774), (92, 12.494666), (91, 11.457912), (59, 11.365786), (85, 10.594591), (66, 10.157406), (73, 9.793003), (74, 9.500913), (58, 9.370901), (84, 8.814832), (93, 8.281558), (38, 7.0878386), (75, 7.085448), (67, 7.061876), (68, 7.022904), (32, 6.492978), (96, 6.366578), (60, 6.283898), (105, 6.1720543), (64, 6.1083446), (30, 6.0496445), (33, 5.8713126), (31, 5.793869), (40, 5.7752466), (0, 5.7042713), (71, 5.691296), (77, 5.5655565), (116, 5.5250587), (126, 5.3674726), (29, 5.3442388), (65, 5.331758), (70, 5.258166), (2, 5.2549467), (123, 5.2481394), (83, 5.244583), (61, 5.2110534), (79, 5.155571), (54, 5.124194), (39, 4.988542), (34, 4.9790974), (1, 4.9214525), (107, 4.8902593), (35, 4.800258), (76, 4.745479), (112, 4.7073054), (69, 4.6913037), (8, 4.623235), (117, 4.5903287), (128, 4.5267286), (57, 4.4213266), (78, 4.39845), (62, 4.3114433), (80, 4.2783093), (111, 4.2106786), (56, 

In [79]:
# 假设长序列为 long_sequence，索引组成的列表为 indices_list
long_sequence = imgsData  # 长序列示例，这里假设长度为100 # 索引组成的列表示例

# 定义每个索引前后元素的数量
window_size = 5

# 提取每个索引前后的元素，组成子序列
sub_sequences = [long_sequence[max(0, index - window_size):index + window_size + 1] for index in indices_list]

for franm in sub_sequences[1]:
    cv2.imshow("im",franm)
    cv2.waitKey(-1)

cv2.destroyAllWindows()


In [94]:

for vi,frames in enumerate(sub_sequences):
    # 逐帧写入视频
    for ii,frame in enumerate(frames):
        os.makedirs(f'result/{vi}',exist_ok=True)
        output_path = f'result/{vi}/{ii}.jpg'
        cv2.imwrite(output_path,frame)


## GUI: 裁切画面

弹出一个窗口供用户可以裁切小鼠活动范围，鼠标拖拽会出现一个绿色的框框，按s保存，按q不保存退出。

In [110]:
import yaml
import cv2
import logging

# 定义全局变量
x_start, y_start, x_end, y_end = 0, 0, 0, 0
cropping = False

def draw_rectangle(event, x, y, flags, param):
    global x_start, y_start, x_end, y_end, cropping

    if event == cv2.EVENT_LBUTTONDOWN:
        x_start, y_start, x_end, y_end = x, y, x, y
        cropping = True

    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping:
            x_end, y_end = x, y

    elif event == cv2.EVENT_LBUTTONUP:
        x_end, y_end = x, y
        cropping = False

# 创建GUI窗口
cv2.namedWindow('Input Image')
cv2.setMouseCallback('Input Image', draw_rectangle)

# 读取输入图像
cap = cv2.VideoCapture(video_path)
ret, image = cap.read()
cap.release()
# image = imgsData[0]

while True:
    # 显示输入图像并绘制矩形框
    clone = image.copy()
    cv2.rectangle(clone, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
    cv2.imshow('Input Image', clone)

    # 按下键盘上的 's' 键保存裁剪的图像
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):
        cropped_image = image[y_start:y_end, x_start:x_end]
        logging.info(f"crop region is [{x_start} to {x_end}] of x axis and [{y_start} to {y_end}] of y axis")
        crop_info = {'x_start': x_start, 'y_start': y_start, 'x_end': x_end, 'y_end': y_end}
        with open('crop_info.yaml', 'w') as yaml_file:
            yaml.dump(crop_info, yaml_file, default_flow_style=False)
        logging.info('裁剪信息已保存到 crop_info.yaml')
        # cv2.imwrite('cropped_image.jpg', cropped_image)
        break

    # 按下键盘上的 'q' 键退出程序
    elif key == ord('q'):
        break

# 关闭窗口
cv2.destroyAllWindows()


使用以下代码来读取YAML文件中的裁剪信息：

In [28]:
import yaml

# 从YAML文件中加载裁剪信息
with open('crop_info.yaml', 'r') as yaml_file:
    crop_info = yaml.safe_load(yaml_file)

# 打印裁剪信息
print("裁剪范围信息：")
print("x_start:", crop_info['x_start'])
print("y_start:", crop_info['y_start'])
print("x_end:", crop_info['x_end'])
print("y_end:", crop_info['y_end'])
# cropped_image = image[y_start:y_end, x_start:x_end]

裁剪范围信息：
x_start: 428
y_start: 31
x_end: 1120
y_end: 576


In [114]:

# 获取当前日期和时间
current_datetime =

# 格式化日期和时间为字符串

print("当前日期和时间的字符串表示形式:", datetime_string)


当前日期和时间的字符串表示形式: 2024-03-25_23:43:36
